<div align="center">
    <h1>DS-210: Programming for Data Science</h1>
    <h1>Lecture 8</h1>
</div>


# 1. Ownership and borrowing in Rust
# 2. Methods in Rust
# 3. Copying instead of moving
# 4. Multiple references in parallel
# 5. Generics
# 6. Useful predefined generic data types



In [6]:
let mut pointer = Box::new(2000);
let pointer2 = Box::new(22);

pointer = pointer2;
println!("vals: {}", pointer);
println!("vals: {}", pointer2);


Error: value assigned to `pointer` is never read

Error: borrow of moved value: `pointer2`

In [8]:
let mut pointer = Box::new(2000);
println!("{:?}", pointer);
//let z = pointer;
//println!("{:?}", z);
drop(pointer);
println!("{:?}", pointer);

2000
2000


## Experiment with passing the pointer around

In [11]:
fn print_content(pointer: Box<i32>) {
    println!("content: {}", *pointer)
}

fn print_value(val: i32){
    println!("content: {}", val)
}

let p = Box::new(123);
print_value(*p);
print_value(*p);
print_content(p); 

content: 123
content: 123
content: 123


In [12]:
let q = Box::new(321);

print_content(q);
print_content(q);

Error: use of moved value: `q`

## What happened: Ownership

* Each value in Rust has a variable that is its **owner**
* Only **one** owner
* When the owner goes out of scope, the value is dropped

In [13]:
fn print_content(pointer:Box<i32>) {
    println!("content: {}", *pointer)
}

let q = Box::new(321);

print_content(q);
print_content(q);

Error: use of moved value: `q`

* First call to `print_content`: `Box::new(321)` is **moved** from `q` to `pointer`
* (if it compiled) at the end of `print_content`:
  * `Box::new(321)` would be dropped
  * its space on the heap deallocated

Second call can't proceed: **the content of `q` is gone**

## More examples of ownership

In [14]:
// won't work, value moved as well
let x = Box::new(123);
println!("x = {}",*x);
let y = x;
println!("x = {}",*x);

Error: borrow of moved value: `x`

Fix our previous example by returning the pointer

In [15]:
fn print_content(pointer:Box<i32>) -> Box<i32> {
    println!("content: {}", *pointer);
    pointer
}

let q = Box::new(321);

let q = print_content(q);
let q = print_content(q);
let q = print_content(q);

content: 321
content: 321
content: 321


## Avoiding moving values a lot: borrowing

In [13]:
#[derive(Debug)]
struct Road {
    intersection_1: u32,
    intersection_2: u32,
    max_speed: u32,
}

// adding a function in the namespace of Road
impl Road {
    // very useful constructor
    fn new(i1:u32,i2:u32,speed:u32) -> Road {
        Road {
            intersection_1: i1,
            intersection_2: i2,
            max_speed: speed,
        }
    }
}

let road = Road::new(13,23,25);
println!("{}",road.max_speed);

25


In [14]:
// checking whether it moves
let another = road;
println!("{}",road.max_speed);

Error: borrow of moved value: `road`

In [17]:
fn display_1(r:Road) {
    println!("{:?}",r);
}

let road = Road::new(101,102,30);
display_1(road);
// Uncomment this one to see ownership behavior
//display_1(road);  

Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }


In [18]:
// This one works due to returning the value
fn display_1(r:Road) -> Road {
    println!("{:?}",r);
    r
}

let mut road = Road::new(101,102,30);
road = display_1(road);
road = display_1(road);

Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }


## Avoiding moving values a lot: borrowing

Read–only reference:
* Reference type becomes `&Type`
* To create: `&value`
* To access content: `*reference`

In [22]:
fn display_2(r:&Road) {
    println!("{:?}", *r); // This is the right one
    println!("{:?}", r); // In this case the compiler does the conversion for you but it may not work all times.
    println!("{:?}", &r); // Ditto here.
}

let road = Road::new(101,102,30);
display_2(&road); // <- have to explicitly create a reference
display_2(&road);

Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }
Road { intersection_1: 101, intersection_2: 102, max_speed: 30 }


## Avoiding moving values a lot: borrowing
Mutable reference:
* Reference type becomes `&mut Type`
* To create: `&mut value`
* To access content: `*reference`


In [23]:
// regular references won't work
fn update_speed(r:&Road, new_speed: u32) {
    // r.max_speed equivalent to (*r).max_speed
    // because Rust is smart
    r.max_speed = new_speed;
} 

Error: cannot assign to `r.max_speed`, which is behind a `&` reference

In [24]:
fn update_speed(r:&mut Road, new_speed: u32) {
    // r.max_speed equivalent to (*r).max_speed
    // because Rust is smart
    r.max_speed = new_speed;
} 

In [25]:
let mut road = Road::new(100,200,30);
display_2(&road);
update_speed(&mut road, 25);
display_2(&road);

Road { intersection_1: 100, intersection_2: 200, max_speed: 30 }
Road { intersection_1: 100, intersection_2: 200, max_speed: 30 }
Road { intersection_1: 100, intersection_2: 200, max_speed: 30 }
Road { intersection_1: 100, intersection_2: 200, max_speed: 25 }
Road { intersection_1: 100, intersection_2: 200, max_speed: 25 }
Road { intersection_1: 100, intersection_2: 200, max_speed: 25 }


## Methods


* We can add functions that are directly associated with structs and enums!
  * Then we could call them: `road.display()` or `road.update_speed(25)`
* How?
  * Put them in the namespace of the type
  * make `self` the first argument

In [26]:
#[derive(Debug)]
struct Road {
    intersection_1: u32,
    intersection_2: u32,
    max_speed: u32,
}

impl Road {
    fn new(i1:u32,i2:u32,speed:u32) -> Road {
        Road {
            intersection_1: i1,
            intersection_2: i2,
            max_speed: speed,
        }
    }
    // note &self: immutable reference
    fn display(&self) {
        println!("{:?}",*self);
    }
}

In [27]:
let mut road = Road::new(1,2,35);
road.display();
&road.display();
(&road).display();

Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }
Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }
Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }


## Methods (continued)

In [28]:
impl Road {
    fn n_update_speed(&self, new_speed:u32) {
        self.max_speed = new_speed;
    }
}

Error: cannot assign to `self.max_speed`, which is behind a `&` reference

In [29]:
impl Road {
    fn update_speed(&mut self, new_speed:u32) {
        self.max_speed = new_speed;
    }
}

In [30]:
road.display();
road.update_speed(25);
road.display();

Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }
Road { intersection_1: 1, intersection_2: 2, max_speed: 25 }


In [31]:
impl Road {
    
    fn this_will_move(self) -> Road {
        self
    }
    
    fn this_will_not_move(&self) -> &Road {
        self
    }
}

In [33]:
fn testme() {
  let r = Road::new(1,2,35);
  let r3 = r.this_will_not_move();
 // let r2 = r.this_will_move();
  r.display();
 // r2.display();
  r3.display();
}
testme();

Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }
Road { intersection_1: 1, intersection_2: 2, max_speed: 35 }


## Methods (summary)

* Make first parameter `self`
* Various options:
  - `self`: move will occur
  - `&self`: self will be immutable reference
  - `&mut self`: self will be mutable reference

## A few more pointer types

In [ ]:
// A few more pointer types for future reference

use std::rc::Rc;
use std::cell::RefCell;

// A reference counted pointer.  Behaves like a box pointer in this example
let mut pointer = Rc::new(12);
let pointer2 : Rc<i32> = Rc::new(22);
println!("sum: {}", *pointer + *pointer2);

// But as we saw you can not assigned box pointers to each other without clone
let mut pointer = Box::new(10);
let mut pointer2 = pointer.clone();
*pointer2 = 22;
println!("Box Vals {} {} ", *pointer, *pointer2);


// But Rc pointers with refcells allow for this, 
// though you have to tell the compiler a lot of information to allow it
let mut pointer = Rc::new(RefCell::new(10));
let mut pointer2 = Rc::new(Rc::clone(&pointer));
*pointer.borrow_mut() = 22;
println!("Ref Vals {} {} ", *pointer.borrow(), *pointer2.borrow());

In [2]:
#[derive(Debug)]
struct BoxSize {
    height: f64,
    width: f64,
    depth: f64,
}

impl BoxSize {
    fn new(height: f64, width: f64, depth: f64)
        -> BoxSize {
        BoxSize{
            height: height, 
            width, // = width: width
            depth, // = depth: depth
        }
    }
}

In [5]:
let xl_box = BoxSize::new(24.0,18.0,24.0);
println!("{:?}", xl_box);
let move_it_here = xl_box;
println!("{:?}", xl_box);

Error: borrow of moved value: `xl_box`

<div align="center">
<img src="uhaul_xl.png" alt="[image of a box]">
</div>

## Cloning

How to make a copy of data?

**Option 1:** Implement yourself

In [6]:
impl BoxSize {
    fn give_me_a_copy(&self) -> BoxSize {
        let BoxSize{height,width,depth} = *self;
        BoxSize{height,width,depth}
    }
}

In [7]:
let box_1 = BoxSize::new(1.1,2.2,3.3);
println!("{:?}",box_1);
let box_2 = box_1.give_me_a_copy();
println!("================");
println!("{:?}",box_1);
println!("{:?}",box_2);
// Also
println!("{:?}",&box_1);
println!("{:?}",&box_2);
// But 
println!("{:p} {:p}", &box_1, &box_2)

BoxSize { height: 1.1, width: 2.2, depth: 3.3 }
BoxSize { height: 1.1, width: 2.2, depth: 3.3 }
BoxSize { height: 1.1, width: 2.2, depth: 3.3 }
BoxSize { height: 1.1, width: 2.2, depth: 3.3 }
BoxSize { height: 1.1, width: 2.2, depth: 3.3 }
0x16b106c68 0x16b106c80


()

## Cloning

**Option 2:** Default cloning (with some extra benefits)
* Use `#[derive(Clone)]` in the definition
* Use method `.clone()` to clone an object

In [8]:
#[derive(Clone,Debug)]
struct CloneablePoint {
    x: f64,
    y: f64,
}

In [9]:
let point_1 = CloneablePoint{x:2.2,y:-1.4};
let point_2 = point_1.clone();
println!("{:?}\n{:?}",point_1,point_2);

CloneablePoint { x: 2.2, y: -1.4 }
CloneablePoint { x: 2.2, y: -1.4 }


Can then be used recursively:

In [10]:
// will work
let tuple_point = (1,CloneablePoint{x:1.1,y:1.1});
let copy_tuple_point = tuple_point.clone();

In [11]:
// won't work
let tuple_box = (1,BoxSize::new(1.1,1.2,1.3));
let copy_tuple_box = tuple_box.clone();

Error: Couldn't automatically determine type of variable `copy_tuple_box`.
Please give it an explicit type.

## Implicit copying

* Works for intergers, floats, booleans, ...
* Also for tuples made of items for which it works
* But not for other types including those that implement clone

In [12]:
let int = 3;
let int_2 = int;
println!("{}\n{}",int,int_2);

3
3


In [13]:
let tuple = (1.2, 3.1);
let tuple_2 = tuple;
println!("{:?}\n{:?}",tuple,tuple_2);

(1.2, 3.1)
(1.2, 3.1)


To make it work: use `#[derive(Copy)]` in the definition
* `(Clone)` needed as well

In [18]:
#[derive(Copy,Clone,Debug)]
enum SearchResult {
    DidntFindIt,
    FoundIt(usize),
}

In [19]:
let result = SearchResult::DidntFindIt;
let will_it_move = result;

println!("{:?}\n{:?}",result,will_it_move);

DidntFindIt
DidntFindIt


## What really happens with `derive(Copy)` and `derive(Clone)`

* Defining a specific method or methods (i.e., `clone`)

* It tells Rust that the type meets specific requirements 
  * they are called a trait
  * to be covered later in class (next lecture)

## Multiple references at once

* useful for when we may want to access the same thing from multiple places
* they can be passed around like values

In [21]:
// auxiliary functions

fn display(x:&i32) {
    println!("{}",*x);    
}

fn double(x:&mut i32) {
    *x *= 2;
}

In [22]:
// two immutable references
let mut integer = 1;
{
    integer = 5;
    let ir = &integer;
    let ir2 = &integer;
    display(ir);
    display(ir2);
    println!("{:p} {:p}", ir, ir2);
};

5
5
0x16b106bcc 0x16b106bcc


## Multiple references at once

* useful for when we may want to access the same thing from multiple places
* they can be passed around like values

In [24]:
// one mutable reference
{
    let mr = &mut integer;
    double(mr);
    display(mr);
};

20


In [25]:
// two mutable references
{
    let mr = &mut integer;
    let mr2 = &mut integer;
    double(mr);
    double(mr2);
};

Error: cannot borrow `integer` as mutable more than once at a time

## Multiple references at once

* useful for when we may want to access the same thing from multiple places
* they can be passed around like values

In [26]:
// immutable and mutable references
{
    let mr = &integer;
    let mr2 = &mut integer;
    display(mr);
    double(mr2);
};

Error: cannot borrow `integer` as mutable because it is also borrowed as immutable

In [27]:
// immutable and mutable references
{
    let ir = &integer;
    display(ir);
    let mr2 = &mut integer;
    double(mr2);
    let ir2 = &integer;
    display(ir2);
};

20
40


Rust can figure out which references no longer used

## Multiple references at once

* useful for when we may want to access the same data from multiple places
* they can be passed around like values

### Rules
* At most one mutable reference at a time
* Multiple immutable references allowed
* No mutable and immutable references at the same time

### How it could be useful
* More clear what is happening
  * Potential early bug detection
* Additional optimizations possible
* Multithreading (running things in parallel):
  * each thread accesses things through references
  * potentially very unpredictable behaviour
    without these rules

## Not covered today: lifetimes
* how long a reference lives
* important for making sure that references passed around are not in conflict
* useful for dealing with some data processing patterns

## New Topic: Avoiding copying code for different types

Python:

```python
def max(x,y):
    return x if x > y else y
```
```python
max(3,2)
```
```
3
```
```python
max(3.1,2.2)
```
```
3.1
```

Very flexible! Any downsides?

* Requires checking each time what types are used
* Runtime penalty

## New Topic: Avoiding copying code for different types

Possible Rust "equivalent": create a copy for each type

In [28]:
fn max_i32(x:i32,y:i32) -> i32 {
    if x > y {x} else {y}
}

max_i32(3,8)

8

In [29]:
fn max_f64(x:f64,y:f64) -> f64 {
    if x > y {x} else {y}
}

max_f64(3.3,8.1)

8.1

Lots of work! Make the compiler do it!

In [23]:
fn max<T>(x:T,y:T) -> T {
        if x > y {x} else {y}
}

Error: binary operation `>` cannot be applied to type `T`

In [30]:
// add info that elements of T are comparable
fn max<T:PartialOrd>(x:T,y:T) -> T {
        if x > y {x} else {y}
}

println!("{}",max(3,8));
println!("{}",max(3.3,8.1));
println!("{}",max('a','b'));

8
8.1
b


## Generics / Generic data types
In other programming languages:<br>
&nbsp;&nbsp;&nbsp;&nbsp;$\bullet$ C++: templates<br>
&nbsp;&nbsp;&nbsp;&nbsp;$\bullet$ Java: generics<br>
&nbsp;&nbsp;&nbsp;&nbsp;$\bullet$ Go: generics<br>
&nbsp;&nbsp;&nbsp;&nbsp;$\bullet$ ML, Haskell: parametric polymorphism

## Use with data types

In [31]:
#[derive(Debug)]
struct Point<T> {
    x: T,
    y: T,
}

In [32]:
let point_int = Point {x: 2, y: 3};
println!("{:?}", point_int);

let point_float = Point {x: 4.2, y: 3.0};
println!("{:?}", point_float);


Point { x: 2, y: 3 }
Point { x: 4.2, y: 3.0 }


Functions and methods for generic data types

In [27]:
impl<T> Point<T> {
    fn create(x:T,y:T) -> Point<T> {
        Point{x,y}
    }
}

In [28]:
let point = Point::create('a','b');
let point2 = Point::<char>::create('c','d');
let point3 : Point<char> = Point::create('c','d');
println!("{:?} {:?} {:?}", point, point2, point3);


Point { x: 'a', y: 'b' } Point { x: 'c', y: 'd' } Point { x: 'c', y: 'd' }


## Use with data types

Implementing a method

In [29]:
impl<T:Copy> Point<T> {
    fn swap(&mut self) {
        let z = self.x;
        self.x = self.y;
        self.y = z;
    }
}

In [30]:
impl<T> Point<T> {
    fn swap2(&mut self) {
        let z = self.x;
        self.x = self.y;
        self.y = z;
    }
}

Error: cannot move out of `self.x` which is behind a mutable reference

Error: cannot move out of `self.y` which is behind a mutable reference

In [31]:
let mut point = Point::create(2,3);
println!("{:?}",point);
point.swap();
println!("{:?}",point);


Point { x: 2, y: 3 }
Point { x: 3, y: 2 }


## Use with data types

Specialized versions for different types

In [32]:
impl Point<i32> {
    fn do_you_use_f64(&self) -> bool {
        false
    }
}

In [33]:
impl Point<f64> {
    fn do_you_use_f64(&self) -> bool {
        true
    }
}


In [34]:
let p_i32 = Point::create(2,3);
p_i32.do_you_use_f64()

false

In [35]:
let p_f64 = Point::create(2.1,3.1);
p_f64.do_you_use_f64()

true

# <font color="red">Useful predefined generic data types</font>


* Generic code
* Method for avoiding copying code
* No runtime penalty: different versions created during compilation

Generic data types:
* Data types (struct/enum) parameterized by types

Two useful predifined types: `Option<T>` and `Result<T, E>`

## Enum `Option<T>`

`Some(T)` or `None`

* Useful for when there may be no output
* Compared to `None` or `null` in other programming languages:
  * Rust forces handling of this case
  
<div align="center">
<img src="null.png" alt="[abc]" >
</div>

In [24]:
fn prime(x:u32) -> bool {
    if x <= 1 { return false;}
    for i in 2..=((x as f64).sqrt() as u32) {
        if x % i == 0 {
            return false;
        }
    } 
    true
}

fn prime_in_range(a:u32,b:u32) -> Option<u32> {
    for i in a..=b {
        if prime(i) {return Some(i);}
    }
    None
}

In [5]:
prime_in_range(890,906)

None

In [25]:
let tmp : Option<u32> = prime_in_range(830,856);
tmp

Some(839)

In [26]:
// extracting the content of Some(...)
if let Some(x) = tmp {
    println!("{}",x);
}
match tmp {
    Some(x) => println!("{}",x),
    None => println!("None"),
};
println!("Another way {}", tmp.unwrap())

839
839
Another way 839


()

In [8]:
// extracting the content of Some(...)
let tmp: Option<u32> = None;
if let Some(x) = tmp {
    println!("{}",x);
}
match tmp {
    Some(x) => println!("{}",x),
    None => println!("{:?}", tmp),
};
// Boom!!!!!
println!("Another way {}", tmp.unwrap())

None


thread '<unnamed>' panicked at 'called `Option::unwrap()` on a `None` value', src/lib.rs:138:32
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::panicking::panic
   3: <core::panic::unwind_safe::AssertUnwindSafe<F> as core::ops::function::FnOnce<()>>::call_once
   4: _run_user_code_5
   5: evcxr::runtime::Runtime::run_loop
   6: evcxr::runtime::runtime_hook
   7: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


## Interesting related fact: Bertrand's postulate

<br>
<br>
<div align="center">
<h4>There is always a prime number in $[k,2k]$.</h4>
    <a> https://en.wikipedia.org/wiki/Prime_number_theorem </a>
</div>

# Enum `Option<T>`: useful methods

Check the variant
* `.is_some() -> bool`
* `.is_none() -> bool`

Get the value in `Some` or terminate with an error
* `.unwrap() -> T`
* `.expect(message) -> T`

Get the value in `Some` or a default value
* `.unwrap_or(default_value:T) -> T`

In [27]:
let x = Some(3);
x.is_some()

true

In [28]:
// let x:Option<u32> = Some(3);
let x = None;
let y:u32 = x.expect("This should have been an integer");
y

thread '<unnamed>' panicked at 'This should have been an integer', src/lib.rs:252:15
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::panicking::panic_display
   3: core::panicking::panic_str
   4: core::option::expect_failed
   5: <core::panic::unwind_safe::AssertUnwindSafe<F> as core::ops::function::FnOnce<()>>::call_once
   6: _run_user_code_24
   7: evcxr::runtime::Runtime::run_loop
   8: evcxr::runtime::runtime_hook
   9: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


In [29]:
let x = Some(3);
// let x = None;
x.unwrap_or(0)

3

More details:
* https://doc.rust-lang.org/std/option/
* https://doc.rust-lang.org/std/option/enum.Option.html

## Enum `Result<T, E>`

`Ok(T)` or `Err(E)`

* Useful when you want to pass a solution or information about an error

In [12]:
fn divide(a:u32,b:u32) -> Result<u32,String> {
    match b {
        0 => Err(String::from("Division by zero")),
        _ => Ok(a / b)
    }
}

In [13]:
divide(3,0)

Err("Division by zero")

In [14]:
divide(2022,3)

Ok(674)

# Enum `Result<T, E>`: useful methods

Check the variant
* `.is_ok() -> bool`
* `.is_err() -> bool`

Get the value in `Ok` or terminate with an error
* `.unwrap() -> T`
* `.expect(message) -> T`

Get the value in `Ok` or a default value
* `.unwrap_or(default_value:T) -> T`

In [15]:
let r1 : Result<i32,()> = Ok(3);
// r1.is_err()
r1.is_ok()

true

In [16]:
r1.unwrap()

3

In [17]:
let r2 : Result<u32,()> = Err(());
let r3 : Result<u32,()> = Ok(123);
println!("r2: {}\nr3: {}",
    r2.unwrap_or(0),
    r3.unwrap_or(0));


r2: 0
r3: 123


More details:
* https://doc.rust-lang.org/std/result/
* https://doc.rust-lang.org/std/result/enum.Result.html

## Read chapters 4, 10 skip 10.3